In [123]:
import pandas as pd
import numpy as np

In [124]:
data_path = r'.\data\mean_substitution.csv'

In [125]:
df = pd.read_csv(data_path, index_col = 'Unnamed: 0')

In [126]:
df['day'] = df['time'].str.split(' ').str[0]

In [127]:
df['time'] = df['time'].str.split(' ').str[1]

In [128]:
df['day_id'] = df['id'] + '_' + df['day']

In [129]:
day_ids = list(set(df['day_id']))

In [134]:
first = {}
last = {}
df_activity = df[df['variable'] == 'activity']

for day_id in day_ids:
    df_day_id = df_activity[df_activity['day_id'] == day_id]
    df_day_id = df_day_id.sort_values(by=['time'])
    sleep = df_day_id[df_day_id['value'] != 0]
    if len(sleep) > 0:
        first[day_id] = int(str(df_day_id[df_day_id['value'] != 0].iloc[0]['time']).split(':')[0])
        last[day_id] = int(str(df_day_id[df_day_id['value'] != 0].iloc[-1]['time']).split(':')[0])
    else:
        first[day_id] = np.nan
        last[day_id] = np.nan

In [136]:
print(set(df['variable']))
averages = ['mood', 'activity', 'circumplex.valence', 'circumplex.arousal']
count = ['appCat.other', 'appCat.finance', 'appCat.social', 'appCat.travel', 'appCat.game', 'appCat.builtin', 'appCat.communication', 'call', 'appCat.unknown', 'appCat.office', 'appCat.weather', 'sms', 'appCat.entertainment', 'screen', 'appCat.utilities']

{'appCat.finance', 'appCat.weather', 'circumplex.valence', 'appCat.office', 'sms', 'screen', 'activity', 'appCat.other', 'appCat.travel', 'appCat.game', 'appCat.entertainment', 'appCat.communication', 'appCat.utilities', 'circumplex.arousal', 'mood', 'appCat.social', 'appCat.builtin', 'appCat.unknown', 'call'}


In [137]:
df_clean = df[['day', 'day_id', 'id']].copy()
df_clean = df_clean.drop_duplicates()

In [138]:
for variable in averages:
    df_var = df[df['variable'] == variable]
    avg_var = df_var.groupby('day_id')['value'].mean()
    df_clean[variable] = df_clean['day_id'].map(avg_var)
    if variable == 'mood':
        moods = avg_var.to_dict()

for variable in count:
    df_var = df[df['variable'] == variable]
    sum_var = df_var.groupby('day_id')['value'].sum()
    df_clean[variable] = df_clean['day_id'].map(sum_var)       


In [139]:
next_mood = {}
dates = sorted(list(set(df_clean['day'])))
for dateid in moods.keys():
    d_i = dateid.split('_')
    d = d_i[1]
    i = d_i[0]
    d_index = dates.index(d)
    if d_index < len(dates) - 1:
        new_d_i = i + '_' + dates[d_index + 1]
        if new_d_i in moods:
            next_mood[dateid] = moods[new_d_i]
        else:
            next_mood[dateid] = np.nan
    else:
        next_mood[dateid] = np.nan

In [140]:
df_clean['target'] = df_clean['day_id'].map(next_mood)

In [141]:
df_clean['wake_up'] = df_clean['day_id'].map(first)

In [142]:
df_clean['sleep'] = df_clean['day_id'].map(last)

In [143]:
df_clean = df_clean.drop_duplicates().sort_values('day_id')

In [157]:
standardize = ['appCat.other', 'appCat.finance', 'appCat.social', 'appCat.travel', 'appCat.game', 'appCat.builtin', 'appCat.communication', 'appCat.unknown', 'appCat.office', 'appCat.weather', 'appCat.entertainment', 'screen', 'appCat.utilities']

In [159]:
for var in standardize:
    df_clean[var] = (df_clean[var]-df_clean[var].mean())/df_clean[var].std()

In [160]:
df_clean.isna().sum()

day                     0
day_id                  0
id                      0
mood                    0
activity                0
circumplex.valence      0
circumplex.arousal      0
appCat.other            0
appCat.finance          0
appCat.social           0
appCat.travel           0
appCat.game             0
appCat.builtin          0
appCat.communication    0
call                    0
appCat.unknown          0
appCat.office           0
appCat.weather          0
sms                     0
appCat.entertainment    0
screen                  0
appCat.utilities        0
target                  0
wake_up                 0
sleep                   0
week_day                0
week_mood               0
dtype: int64

In [161]:
df_clean.dropna(subset=['target'], inplace=True)

In [162]:
df_clean['activity'] = df_clean['activity'].fillna(df_clean['activity'].mean())

In [163]:
df_clean['wake_up'] = df_clean['wake_up'].fillna(df_clean['wake_up'].median())

In [164]:
df_clean['sleep'] = df_clean['sleep'].fillna(df_clean['sleep'].median())

In [165]:
df_clean = df_clean.fillna(0)

In [166]:
df_clean['week_day'] = pd.to_datetime(df_clean['day']).dt.dayofweek

In [167]:
def week_mood(df):
    result = []

    for i in range(len(df)):
        current_id = df.iloc[i]['id']
        above = df.iloc[:i+1]
        same_id_above = above[above['id'] == current_id]
        week = same_id_above.tail(7)
        avg = week['mood'].mean() if not week.empty else None
        result.append(avg)

    df['week_mood'] = result
    return df

df_clean = week_mood(df_clean)
print(df_clean)

                   day              day_id       id  mood  activity  \
Unnamed: 0                                                            
1           2014-02-26  AS14.01_2014-02-26  AS14.01  6.25  0.119878   
8           2014-03-21  AS14.01_2014-03-21  AS14.01  6.20  0.134050   
13          2014-03-22  AS14.01_2014-03-22  AS14.01  6.40  0.236880   
18          2014-03-23  AS14.01_2014-03-23  AS14.01  6.80  0.142741   
23          2014-03-24  AS14.01_2014-03-24  AS14.01  6.00  0.078961   
...                ...                 ...      ...   ...       ...   
5618        2014-05-26  AS14.33_2014-05-26  AS14.33  5.40  0.115201   
5623        2014-05-27  AS14.33_2014-05-27  AS14.33  6.20  0.012704   
5628        2014-05-28  AS14.33_2014-05-28  AS14.33  8.20  0.103301   
5633        2014-05-29  AS14.33_2014-05-29  AS14.33  7.00  0.169354   
5636        2014-05-30  AS14.33_2014-05-30  AS14.33  6.80  0.192901   

            circumplex.valence  circumplex.arousal  appCat.other  \
Unnamed:

In [168]:
df_classification = df_clean.copy()

In [169]:
df_classification['target'] = df_classification['target'].round().astype(int)

In [170]:
df_classification.to_csv('features_classification.csv', index=True)

In [171]:
df_clean.to_csv('features_regression.csv', index=True)